In [1]:
import pandas as pd
import numpy as np
import re
import os
from ast import literal_eval
from sklearn.preprocessing import StandardScaler, MinMaxScaler
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline


In [23]:
pickle_location = r'/Users/louishagenbucher/Documents/GitHub/mtg_nlp_price_prediction/d:\temp\pickle_file'
df = pd.read_pickle(pickle_location)


In [24]:
df.head()

,name,cmc,B,C,G,N,R,U,W,legendary,...,rarity_rare,rarity_special,rarity_uncommon,released_at,booster,types_embedding,sub_type_embedding,keywords_embedding,text_embeddings,usd
0,+2 Mace,2.0,0,0,0,0,0,0,1,0,...,0.0,0.0,0.0,2021-07-23,1,"[-0.0073298556, 0.0012392023, -0.0074606393, -...","[-0.026635716, 0.05333321, -0.01776248, -0.073...","[-0.103732154, 0.15253988, -0.024377218, -0.01...","[[-0.43040189146995544, -0.4970492124557495, 0...",0.02
1,Aarakocra Sneak,4.0,0,0,0,0,0,1,0,0,...,0.0,0.0,0.0,2022-06-10,1,"[-0.00039087355, 0.00023514114, 0.007447564, 0...","[-0.091230124, 0.26011455, -0.047625706, -0.39...","[-0.19166172, 0.2589344, -0.032022934, -0.0208...","[[-0.16684836149215698, -0.23930270969867706, ...",0.06
2,Abaddon the Despoiler,5.0,1,0,0,0,1,1,0,1,...,0.0,0.0,0.0,2022-10-07,0,"[-0.00039087355, 0.00023514114, 0.007447564, 0...","[-0.090878814, 0.22455463, -0.035121176, -0.32...","[-0.18475257, 0.2746542, -0.039448958, -0.0275...","[[-0.23658572137355804, -0.27686968445777893, ...",2.81
3,Abandoned Outpost,0.0,0,0,0,1,0,0,0,0,...,0.0,0.0,0.0,2001-10-01,1,"[-0.008701766, 0.0022358345, 0.001184414, -0.0...","[-0.00053622725, 0.00023643136, 0.0051033497, ...","[-0.00053622725, 0.00023643136, 0.0051033497, ...","[[-0.34767621755599976, -0.5046898722648621, -...",0.14
4,Abandoned Sarcophagus,3.0,0,1,0,0,0,0,0,0,...,1.0,0.0,0.0,2020-04-17,0,"[-0.0073298556, 0.0012392023, -0.0074606393, -...","[-0.00053622725, 0.00023643136, 0.0051033497, ...","[-0.00053622725, 0.00023643136, 0.0051033497, ...","[[-0.037254445254802704, -0.20066097378730774,...",0.09


In [25]:
df['types_embedding'][0].shape

(100,)

In [26]:
df['sub_type_embedding'][0].shape

(100,)

In [27]:
df['keywords_embedding'][0].shape

(100,)

In [28]:
df['text_embeddings']= df['text_embeddings'].apply(np.array)

In [29]:
df['text_embeddings'][0].shape

(1, 768)

In [30]:
def reshape_array(arr):
    return arr.reshape(768,)

df['text_embeddings'] = df['text_embeddings'].apply(reshape_array)
df['text_embeddings'][0].shape

(768,)

In [31]:
embedding_columns = ['types_embedding', 'sub_type_embedding', 'keywords_embedding', 'text_embeddings']
df['combined_embedding'] = df[embedding_columns].apply(np.concatenate, axis=1)

/var/folders/tm/jmhp8xvj4_d4hh_01cf1jt_r0000gn/T/ipykernel_14131/928793170.py:2: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  df['combined_embedding'] = df[embedding_columns].apply(np.concatenate, axis=1)


In [32]:
df.combined_embedding[0].shape

(1068,)

In [33]:
df.columns

Index(['name', 'cmc', 'B', 'C', 'G', 'N', 'R', 'U', 'W', 'legendary', 'power',
       'toughness', 'reserved', 'rarity_common', 'rarity_mythic',
       'rarity_rare', 'rarity_special', 'rarity_uncommon', 'released_at',
       'booster', 'types_embedding', 'sub_type_embedding',
       'keywords_embedding', 'text_embeddings', 'usd', 'combined_embedding'],
      dtype='object')

In [34]:
df = df[['name','B', 'C', 'G', 'N', 'R', 'U', 'W', 'legendary',
         'reserved', 'rarity_common', 'rarity_mythic',
       'rarity_rare', 'rarity_special', 'rarity_uncommon',
       'booster', 'cmc' ,'power', 'toughness',
       'combined_embedding', 'released_at', 'usd']]

In [35]:
df.sample(1)

,name,B,C,G,N,R,U,W,legendary,reserved,...,rarity_rare,rarity_special,rarity_uncommon,booster,cmc,power,toughness,combined_embedding,released_at,usd
506,Altar's Light,0,0,0,0,0,0,1,0,0,...,0.0,0.0,1.0,1,4.0,0,0,"[-0.008642567321658134, 0.003678044071421027, ...",2003-10-02,0.1


In [36]:
df['released_at'] = pd.to_datetime(df['released_at'])
reference_date = df['released_at'].min()
df['days_since_release'] = (df['released_at'] - reference_date).dt.days
df.drop(columns='released_at', inplace=True)

In [37]:
df.columns

Index(['name', 'B', 'C', 'G', 'N', 'R', 'U', 'W', 'legendary', 'reserved',
       'rarity_common', 'rarity_mythic', 'rarity_rare', 'rarity_special',
       'rarity_uncommon', 'booster', 'cmc', 'power', 'toughness',
       'combined_embedding', 'usd', 'days_since_release'],
      dtype='object')

In [38]:
df = df[['name', 'B', 'C', 'G', 'N', 'R', 'U', 'W', 'legendary', 'reserved',
       'rarity_common', 'rarity_mythic', 'rarity_rare', 'rarity_special',
       'rarity_uncommon', 'booster', 'cmc', 'power', 'toughness','days_since_release',
       'combined_embedding', 'usd']]
df.sample(1)

,name,B,C,G,N,R,U,W,legendary,reserved,...,rarity_rare,rarity_special,rarity_uncommon,booster,cmc,power,toughness,days_since_release,combined_embedding,usd
21724,Tainted Indulgence,1,0,0,0,0,1,0,0,0,...,0.0,0.0,1.0,1,2.0,0,0,10376,"[-0.008642567321658134, 0.003678044071421027, ...",0.19


In [39]:
df['power'] = pd.to_numeric(df['power'], errors='coerce')
df['toughness'] = pd.to_numeric(df['toughness'], errors='coerce')
index = df[df['power'].isna() | df['toughness'].isna()][['power', 'toughness']].index
df[df['power'].isna() | df['toughness'].isna()][['power', 'toughness']]

,power,toughness
24,NaN,NaN
27,NaN,NaN
129,NaN,NaN
143,NaN,4.0
178,NaN,NaN
...,...,...
24860,NaN,3.0
25008,2.0,NaN
25130,NaN,NaN
25321,NaN,NaN


In [41]:
df['power'] = df['power'].fillna(-1).astype(int)
df['toughness'] = df['toughness'].fillna(-1).astype(int)

In [44]:
df.loc[index,['power', 'toughness']].head()

,power,toughness
24,-1,-1
27,-1,-1
129,-1,-1
143,-1,4
178,-1,-1


In [43]:
X = df.drop(columns=['name', 'usd'])
y = df.usd
numerical_features = ['cmc', 'power', 'toughness', 'days_since_release']
numerical_transformer = Pipeline(steps=[
    ('scaler', StandardScaler())
])
# to remember to preprocess when taking an input we would use ohe here
categorical_features = ['rarity_common', 'rarity_mythic', 'rarity_rare', 'rarity_special', 'rarity_uncommon', 'booster']
preprocessor = ColumnTransformer(
    transformers=[
        ('num', numerical_transformer, numerical_features),
        ('cat', 'passthrough', categorical_features),

    ])
X_transformed = preprocessor.fit_transform(X)